In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
xrange = range
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import os

#pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', 200)

print("The working directory was: {0}".format(os.getcwd()))
os.chdir("C:/hjBoralex/etl/gitcwd")
print("The current working directory is: {0}".format(os.getcwd()))

The working directory was: C:\hjBoralex\etl\gitcwd\etls_assets_hedges
The current working directory is: C:\hjBoralex\etl\gitcwd


In [2]:
cwdpath = "C:/Users/hermann.ngayap/Desktop/data/market/markets/in/"

In [3]:
prod = pd.read_excel(cwdpath + "template_prod_id.xlsx", sheet_name="prod")
prod_perc = pd.read_excel(cwdpath + "template_prod_id.xlsx", sheet_name="prod_perc")
mean_perc = pd.read_excel(cwdpath + "template_prod_id.xlsx", sheet_name="mean_perc")


# df_asset = pd.read_excel(cwdpath + "template_asset_vmr_planif.xlsx")
# df_asset = df_asset.loc[df_asset["en_planif"] == "Non"]

# df_planif = pd.read_excel(cwdpath + "template_asset_vmr_planif.xlsx")
# df_planif = df_planif.loc[df_planif["en_planif"] == "Oui"]

df_hedge = pd.read_excel(cwdpath + "template_hedge.xlsx")
df_hedge_vmr = df_hedge.loc[df_hedge["en_planif"] == "Non"]
df_hedge_planif = df_hedge.loc[df_hedge["en_planif"] == "Oui"]

# HEDGE VMR

In [4]:
df_OA = df_hedge_vmr[["projet_id", "type_hedge", "date_debut", "date_fin", "date_dementelement"]]
df_OA = df_OA.loc[df_OA["type_hedge"] == "OA"]
print(df_OA.shape)

#To extract cod date from asset template 
df_CR = df_hedge_vmr[["projet_id", "type_hedge", "date_debut", "date_fin", "date_dementelement"]]
df_CR = df_CR.loc[(df_CR["type_hedge"] != "OA") & (df_CR["type_hedge"] != "PPA")]
print(df_CR.shape)

df_PPA = df_hedge_vmr[["projet_id", "type_hedge", "date_debut", "date_fin", "date_dementelement"]]
df_PPA = df_PPA.loc[df_PPA["type_hedge"] == "PPA"]
print(df_PPA.shape)

(67, 5)
(21, 5)
(8, 5)


## HEDGE OA

In [5]:
#To extract cod date from hedge date with hedge type = OA 
df_OA = df_hedge_vmr[["hedge_id", "projet_id", "type_hedge", "date_debut", "date_fin", "date_dementelement", "pct_couverture"]]
df_OA = df_OA.loc[df_OA["type_hedge"] == "OA"]
#To merge cod data and prod data
df_OA = df_OA.merge(prod, on='projet_id')

#Hedge perct
df_OA["p50"] = df_OA["p50"] * df_OA["pct_couverture"]
df_OA["p90"] = df_OA["p90"] * df_OA["pct_couverture"]

#To determine the number of asset under OA
n_OA = len(df_OA.loc[df_OA["type_hedge"] == "OA"])

#rename prod label with projet_id
prod_perc = prod_perc.rename(columns=prod.set_index('projet')['projet_id'])

#To define a dict containing prod percentage  
d_prod_perc = prod_perc.to_dict()

start_date = pd.to_datetime(["01-01-2022"] * n_OA)
d = pd.DataFrame()
for i in range(0,84):
    df_buffer = df_OA 
    df_buffer["date"] = start_date
    l_p50 = []
    l_p90 = []
    for elm in df_buffer.projet_id:
        try: 
            l_p50.append(-d_prod_perc[elm][start_date.month[0]-1]*float(df_buffer[df_buffer.projet_id==elm]["p50"].values[0]))
        except:
            l_p50.append("NA")
        try:
            l_p90.append(-d_prod_perc[elm][start_date.month[0]-1]*float(df_buffer[df_buffer.projet_id==elm]["p90"].values[0]))
        except:
            l_p90.append("NA")
    df_buffer["p50_adj"] = l_p50
    df_buffer["p90_adj"] = l_p90
    d = pd.concat([d, df_buffer],axis=0)
    start_date = start_date + pd.DateOffset(months=1)
    
#To convert date
d["date_debut"] = pd.to_datetime(d["date_debut"])
d["date"] = pd.to_datetime(d["date"])
d["date_fin"] = pd.to_datetime(d["date_fin"])
d["date_dementelement"] = pd.to_datetime(d["date_dementelement"])

d['année'] = d['date'].dt.year
d['trim'] = d['date'].dt.to_period('Q').dt.strftime("Q%q-%y")
d['mois'] = d['date'].dt.month

d = d[['hedge_id', 'projet_id', 'projet', 'date_debut', 'date', 'date_fin', "date_dementelement", 'année', 'trim', 'mois', 'p50_adj', 'p90_adj']]


#To remove p50 p90 based on date_cod
#Condition:date column is less (in total seconds) than first date for each projet_id's first cod value
cond=((d['date'] - d.groupby(['projet_id', 'hedge_id'])['date_debut'].transform('first')).dt.total_seconds())<0
d['p50_adj'] = np.where(cond,'', d['p50_adj'])
d['p90_adj'] = np.where(cond,'', d['p90_adj'])
#To remove p50 p90 based on date_fin
cond_2=((d['date'] - d.groupby(['projet_id', 'hedge_id'])['date_fin'].transform('first')).dt.total_seconds())>0
d['p50_adj'] = np.where(cond_2, '', d['p50_adj'])
d['p90_adj'] = np.where(cond_2, '', d['p90_adj'])

#To reset index
d.reset_index(inplace=True, drop=True)

d = d.assign(rw_id=[1 + i for i in xrange(len(d))])[['rw_id'] + d.columns.tolist()]
d = d[['rw_id', 'hedge_id', 'projet_id', 'projet', 'année', 'trim', 'mois', 'p50_adj', 'p90_adj']]
#To export results as a data frame
d.to_excel(cwdpath + 'hedge_OA.xlsx', index=False, float_format="%.5f")

In [6]:
d.head()

,rw_id,hedge_id,projet_id,projet,année,trim,mois,p50_adj,p90_adj
0,1,5,ART1,Artois 1,2022,Q1-22,1,-3958.7547507719996,-3709.945818864
1,2,6,ART2,Artois 2,2022,Q1-22,1,-2838.87824925,-2660.453894145
2,3,7,AVI1,Avignonet I,2022,Q1-22,1,,
3,4,8,AVI2,Avignonet II,2022,Q1-22,1,-1050.354,-971.082
4,5,11,BASS,Bassigny,2022,Q1-22,1,-2306.8498935999996,-2150.0736873359997


In [7]:
print(df_OA.loc[df_OA['projet_id'] == 'AVI2'])
df_temp = d.loc[d['projet_id'] == 'AVI2']
df_temp.head(10)

   hedge_id projet_id type_hedge date_debut   date_fin date_dementelement  \
3         8      AVI2         OA 2008-12-29 2023-12-28                NaT   

   pct_couverture        projet      p50     p90       date  p50_adj  p90_adj  
3             1.0  Avignonet II  10600.0  9800.0 2028-12-01 -1007.53  -931.49  


,rw_id,hedge_id,projet_id,projet,année,trim,mois,p50_adj,p90_adj
3,4,8,AVI2,Avignonet II,2022,Q1-22,1,-1050.354,-971.082
70,71,8,AVI2,Avignonet II,2022,Q1-22,2,-968.098,-895.034
137,138,8,AVI2,Avignonet II,2022,Q1-22,3,-1085.122,-1003.226
204,205,8,AVI2,Avignonet II,2022,Q2-22,4,-974.2460000000001,-900.7180000000001
271,272,8,AVI2,Avignonet II,2022,Q2-22,5,-939.796,-868.868
338,339,8,AVI2,Avignonet II,2022,Q2-22,6,-682.216,-630.728
405,406,8,AVI2,Avignonet II,2022,Q3-22,7,-607.486,-561.638
472,473,8,AVI2,Avignonet II,2022,Q3-22,8,-599.854,-554.582
539,540,8,AVI2,Avignonet II,2022,Q3-22,9,-733.52,-678.16
606,607,8,AVI2,Avignonet II,2022,Q4-22,10,-998.414,-923.062


## HEDGE CR

In [8]:
#To extract cod date from hedge date with hedge type = CR
df_CR = df_hedge_vmr[["hedge_id", "projet_id", "type_hedge", "date_debut", "date_fin", "date_dementelement", "pct_couverture"]]
df_CR = df_CR.loc[(df_CR["type_hedge"] != "OA") & (df_CR["type_hedge"] != "PPA")]
#To merge cod data and prod data
df_CR = df_CR.merge(prod, on='projet_id')
#hedge percentage
df_CR["p50"] = df_CR["p50"] * df_CR["pct_couverture"]
df_CR["p90"] = df_CR["p90"] * df_CR["pct_couverture"]

#To determine the number of asset under CR
n_CR = len(df_CR.loc[(df_CR["type_hedge"] != "OA") & (df_CR["type_hedge"] != "PPA")])

#rename prod label with projet_id
prod_perc = prod_perc.rename(columns=prod.set_index('projet')['projet_id'])

#To define a dict containing prod percentage  
d_prod_perc = prod_perc.to_dict()

start_date = pd.to_datetime(["01-01-2022"] * n_CR)
d2 = pd.DataFrame()
for i in range(0,84):
    df_buffer = df_CR 
    df_buffer["date"] = start_date
    l_p50 = []
    l_p90 = []
    for elm in df_buffer.projet_id:
        try: 
            l_p50.append(-d_prod_perc[elm][start_date.month[0]-1]*float(df_buffer[df_buffer.projet_id==elm]["p50"].values[0]))
        except:
            l_p50.append("NA")
        try:
            l_p90.append(-d_prod_perc[elm][start_date.month[0]-1]*float(df_buffer[df_buffer.projet_id==elm]["p90"].values[0]))
        except:
            l_p90.append("NA")
    df_buffer["p50_adj"] = l_p50
    df_buffer["p90_adj"] = l_p90
    d2 = pd.concat([d2, df_buffer],axis=0)
    start_date = start_date + pd.DateOffset(months=1)
    
#To create new columns
d2["date_debut"] = pd.to_datetime(d2["date_debut"])
d2["date"] = pd.to_datetime(d2["date"])
d2["date_fin"] = pd.to_datetime(d2["date_fin"])
d2["date_dementelement"] = pd.to_datetime(d2["date_dementelement"])

d2['année'] = d2['date'].dt.year
d2['trim'] = d2['date'].dt.to_period('Q').dt.strftime("Q%q-%y")
d2['mois'] = d2['date'].dt.month
d2 = d2[['hedge_id', 'projet_id', 'projet', 'date_debut', 'date', 'date_fin', 'date_dementelement','année', 'trim', 'mois', 'p50_adj', 'p90_adj']]


#To remove p50 p90 based on date_debut
#Condition:date column is less (in total seconds) than first date for each projet_id's first cod value
cond=((d2['date'] - d2.groupby(['projet_id', 'hedge_id'])['date_debut'].transform('first')).dt.total_seconds())<0
d2['p50_adj'] = np.where(cond,'', d2['p50_adj'])
d2['p90_adj'] = np.where(cond,'', d2['p90_adj'])
#To remove p50 p90 based on date_fin
cond_2=((d2['date'] - d2.groupby(['projet_id', 'hedge_id'])['date_fin'].transform('first')).dt.total_seconds())>0
d2['p50_adj'] = np.where(cond_2, '', d2['p50_adj'])
d2['p90_adj'] = np.where(cond_2, '', d2['p90_adj'])

#To reset index
d2.reset_index(inplace=True, drop=True)
d2 = d2.assign(rw_id=[1 + i for i in xrange(len(d2))])[['rw_id'] + d2.columns.tolist()]
d2 = d2[['rw_id', 'hedge_id', 'projet_id', 'projet', 'année', 'trim', 'mois', 'p50_adj', 'p90_adj']]
#To export results as a data frame
d2.to_excel(cwdpath + 'hedge_CR.xlsx', index=False, float_format="%.5f")

In [9]:
print(df_CR.loc[df_CR['projet_id'] == 'GO02'])
df_temp = d2.loc[d2['projet_id'] == 'GO02']
df_temp.head(20)

    hedge_id projet_id type_hedge date_debut   date_fin date_dementelement  \
15        69      GO02       CR17 2022-06-01 2042-05-31                NaT   

    pct_couverture                                 projet      p50      p90  \
15             1.0  Repowering Evits et Josaphats (EEJ-R)  42100.0  36800.0   

         date  p50_adj  p90_adj  
15 2028-12-01  -5052.0  -4416.0  


,rw_id,hedge_id,projet_id,projet,année,trim,mois,p50_adj,p90_adj
15,16,69,GO02,Repowering Evits et Josaphats (EEJ-R),2022,Q1-22,1,,
36,37,69,GO02,Repowering Evits et Josaphats (EEJ-R),2022,Q1-22,2,,
57,58,69,GO02,Repowering Evits et Josaphats (EEJ-R),2022,Q1-22,3,,
78,79,69,GO02,Repowering Evits et Josaphats (EEJ-R),2022,Q2-22,4,,
99,100,69,GO02,Repowering Evits et Josaphats (EEJ-R),2022,Q2-22,5,,
120,121,69,GO02,Repowering Evits et Josaphats (EEJ-R),2022,Q2-22,6,-2273.4,-1987.2
141,142,69,GO02,Repowering Evits et Josaphats (EEJ-R),2022,Q3-22,7,-2441.8,-2134.4
162,163,69,GO02,Repowering Evits et Josaphats (EEJ-R),2022,Q3-22,8,-2231.2999999999997,-1950.3999999999999
183,184,69,GO02,Repowering Evits et Josaphats (EEJ-R),2022,Q3-22,9,-2652.3,-2318.4
204,205,69,GO02,Repowering Evits et Josaphats (EEJ-R),2022,Q4-22,10,-3620.6,-3164.7999999999997


In [ ]:
# <!-- ## HEDGE TO MODIFY

# We have to adjust p50 and p90 of 4 parcs:

# * GO02:Repowering Evits et Josaphats (EEJ-R)
# * GO03:Repowering Remise de Reclainville (RCL-R)
# * GO01:Bois des Fontaines (BDF)
# * KEI3:Repowering Bougainville (BGV-R) -->

In [55]:
# #To calculate the modified P50 and p90 from 2023 to 2028 
# from datetime import datetime
# start_date = pd.to_datetime(["01-01-2023"]*4)
# d = pd.DataFrame()
# for i in range(0, 72):
#     df_buffer = df4.copy() 
#     df_buffer["date"] = start_date
#     l_p50 = []
#     l_p90 = []
#     for elm in df_buffer.projet_id:
#         try: 
#             l_p50.append(-d_prod_perc[elm][start_date.month[0]-1]*float(df_buffer[df_buffer.projet_id==elm]["p50"].values[0]))
#         except:
#             l_p50.append("NA")
#         try:
#             l_p90.append(-d_prod_perc[elm][start_date.month[0]-1]*float(df_buffer[df_buffer.projet_id==elm]["p90"].values[0]))
#         except:
#             l_p90.append("NA")
#     df_buffer["p50_md"] = l_p50
#     df_buffer["p90_md"] = l_p90
#     d = pd.concat([d,df_buffer],axis=0)
#     start_date = start_date + pd.DateOffset(months=1)

# d['année'] = d['date'].dt.year
# d['mois'] = d['date'].dt.month
# d.rename(columns = {'projet':'projet_md'}, inplace = True)
# d = d[['projet_id', 'projet_md', 'date', 'année', 'mois', 'p50_md', 'p90_md']]

# #To export modified p50, p90 as excel file
# d.to_excel(marketpath + 'hedge_CR_modified.xlsx', index=False)

# df5 = pd.read_excel(marketpath + 'hedge_CR_modified.xlsx')
# df6 = pd.read_excel(marketpath + 'hedge_CR_to_modify.xlsx')

# df = pd.merge(df6, df5, how="left", on=['projet_id', 'année', 'mois'])

# #To modify old p50 p90 with modified p50 p90
# df.loc[(df['projet'] == df['projet_md']) & (df['année'] == df['date'].dt.year) & (df['mois'] == df['date'].dt.month), 'p50_adj'] = df['p50_md']
# df.loc[(df['projet'] == df['projet_md']) & (df['année'] == df['date'].dt.year) & (df['mois'] == df['date'].dt.month), 'p90_adj'] = df['p90_md']

# #To select columns
# df = df[['ID', 'projet_id', 'projet', 'année', 'mois', 'p50_adj', 'p90_adj']]
# #To export as excel file
# df.to_excel(marketpath + "hedge_CR.xlsx", index=False)

In [22]:
# print(df_CR.loc[df_CR['projet_id'] == 'GO02'])
# df_temp = df.loc[df['projet_id'] == 'GO02']
# df_temp.head(20)

## HEDGE PPA

In [10]:
#To extract cod date from hedge date with hedge type = OA 
df_PPA = df_hedge_vmr[["hedge_id", "projet_id", "type_hedge", "date_debut", "date_fin", "date_dementelement", "pct_couverture"]]
df_PPA = df_PPA.loc[df_PPA["type_hedge"] == "PPA"]
#To merge cod data and prod data
df_PPA = df_PPA.merge(prod, on='projet_id')
#Hedge perct
df_PPA["p50"] = df_PPA["p50"] * df_PPA["pct_couverture"]
df_PPA["p90"] = df_PPA["p90"] * df_PPA["pct_couverture"]

#To determine the number of asset under OA
n_PPA = len(df_PPA.loc[df_PPA["type_hedge"] == "PPA"])

#rename prod label with projet_id
prod_perc = prod_perc.rename(columns=prod.set_index('projet')['projet_id'])

#To define a dict containing prod percentage  
d_prod_perc = prod_perc.to_dict()

start_date = pd.to_datetime(["01-01-2022"] * n_PPA)
d3 = pd.DataFrame()
for i in range(0,84):
    df_buffer = df_PPA 
    df_buffer["date"] = start_date
    l_p50 = []
    l_p90 = []
    for elm in df_buffer.projet_id:
        try: 
            l_p50.append(-d_prod_perc[elm][start_date.month[0]-1]*float(df_buffer[df_buffer.projet_id==elm]["p50"].values[0]))
        except:
            l_p50.append("NA")
        try:
            l_p90.append(-d_prod_perc[elm][start_date.month[0]-1]*float(df_buffer[df_buffer.projet_id==elm]["p90"].values[0]))
        except:
            l_p90.append("NA")
    df_buffer["p50_adj"] = l_p50
    df_buffer["p90_adj"] = l_p90
    d3 = pd.concat([d3, df_buffer],axis=0)
    start_date = start_date + pd.DateOffset(months=1)
    
#To convert date
d3["date_debut"] = pd.to_datetime(d3["date_debut"])
d3["date"] = pd.to_datetime(d3["date"])
d3["date_fin"] = pd.to_datetime(d3["date_fin"])
d3["date_dementelement"] = pd.to_datetime(d3["date_dementelement"])

d3['année'] = d3['date'].dt.year
d3['trim'] = d3['date'].dt.to_period('Q').dt.strftime("Q%q-%y")
d3['mois'] = d3['date'].dt.month
d3 = d3[['hedge_id', 'projet_id', 'projet', 'date_debut', 'date_fin', 'date', "date_dementelement", 'année', 'trim', 'mois', 'p50_adj', 'p90_adj']]


#To remove p50 p90 based on date_cod
#Condition:date column is less (in total seconds) than first date for each projet_id's first cod value
cond=((d3['date'] - d3.groupby(['projet_id', 'hedge_id'])['date_debut'].transform('first')).dt.total_seconds())<0
d3['p50_adj'] = np.where(cond,'', d3['p50_adj'])
d3['p90_adj'] = np.where(cond,'', d3['p90_adj'])
#To remove p50 p90 based on date_dementelement
cond_2=((d3['date'] - d3.groupby(['projet_id', 'hedge_id'])['date_fin'].transform('first')).dt.total_seconds())>0
d3['p50_adj'] = np.where(cond_2, '', d3['p50_adj'])
d3['p90_adj'] = np.where(cond_2, '', d3['p90_adj'])

#To reset index
d3.reset_index(inplace=True, drop=True)

d3 = d3.assign(rw_id=[1 + i for i in xrange(len(d3))])[['rw_id'] + d3.columns.tolist()]
d3 = d3[['rw_id', 'hedge_id', 'projet_id', 'projet', 'année', 'trim', 'mois', 'p50_adj', 'p90_adj']]
#To export results as a data frame
d3.to_excel(cwdpath + 'hedge_PPA.xlsx', index=False, float_format="%.5f")

In [11]:
print(df_PPA.loc[df_PPA['projet_id'] == 'NIBA'])
df_temp = d3.loc[d3['projet_id'] == 'NIBA']
df_temp.tail()

   hedge_id projet_id type_hedge date_debut   date_fin date_dementelement  \
6        56      NIBA        PPA 2021-01-01 2023-12-31                NaT   

   pct_couverture projet          p50          p90       date      p50_adj  \
6             1.0  Nibas  17700.00026  15980.57166 2028-12-01 -2211.615032   

       p90_adj  
6 -1996.772429  


,rw_id,hedge_id,projet_id,projet,année,trim,mois,p50_adj,p90_adj
638,639,56,NIBA,Nibas,2028,Q3-28,8,,
646,647,56,NIBA,Nibas,2028,Q3-28,9,,
654,655,56,NIBA,Nibas,2028,Q4-28,10,,
662,663,56,NIBA,Nibas,2028,Q4-28,11,,
670,671,56,NIBA,Nibas,2028,Q4-28,12,,


In [12]:
df_1=pd.read_excel(cwdpath + 'hedge_OA.xlsx', usecols=['hedge_id', 'projet_id', 'projet', 'année', 'trim', 'mois', 'p50_adj', 'p90_adj'])
df_2=pd.read_excel(cwdpath + 'hedge_CR.xlsx', usecols=['hedge_id', 'projet_id', 'projet', 'année', 'trim', 'mois', 'p50_adj', 'p90_adj'])
df_3=pd.read_excel(cwdpath + 'hedge_PPA.xlsx', usecols=['hedge_id', 'projet_id', 'projet', 'année', 'trim','mois', 'p50_adj', 'p90_adj'])

In [13]:
print(df_1.shape)
print(df_2.shape)
print(df_3.shape)

(5628, 8)
(1764, 8)
(672, 8)


In [14]:
frames = [df_1, df_2, df_3]
p50_p90_hedge_vmr =pd.concat(frames)
p50_p90_hedge_vmr.reset_index(drop=True, inplace=True)

p50_p90_hedge_vmr = p50_p90_hedge_vmr.assign(rw_id=[1 + i for i in xrange(len(p50_p90_hedge_vmr))])[['rw_id'] + p50_p90_hedge_vmr.columns.tolist()]
p50_p90_hedge_vmr = p50_p90_hedge_vmr[['rw_id', 'hedge_id', 'projet_id', 'projet', 'année', 'trim', 'mois', 'p50_adj', 'p90_adj']]
p50_p90_hedge_vmr.to_excel(cwdpath + "p50_p90_hedge_vmr.xlsx", index=False, float_format="%.5f")